In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import quote
from tqdm import tqdm

In [2]:
url = f'https://kin.naver.com/search/list.naver?query={quote("서울 월세")}&page=1'
driver = webdriver.Chrome()
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
time.sleep(3)

The chromedriver version (117.0.5938.88) detected in PATH at C:\Users\human\Downloads\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (118.0.5993.88); currently, chromedriver 118.0.5993.70 is recommended for chrome 118.*, so it is advised to delete the driver in PATH and retry


In [3]:
lis = soup.select('.basic1 > li')
len(lis)

10

In [4]:
li = lis[6]
sub_url = li.select_one('a._searchListTitleAnchor')['href']
sub_url

'https://kin.naver.com/qna/detail.naver?d1id=4&dirId=4020202&docId=455469254&qb=7ISc7Jq4IOyblOyEuA==&enc=utf8&section=kin&rank=7&search_sort=0&spq=0'

In [5]:
driver.get(sub_url)
time.sleep(2)

In [22]:
sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
questions = sub_soup.select('#content > div.question-content > div > div.c-heading._questionContentsArea.c-heading--default-old > div.c-heading__content')
divs = sub_soup.select('#answer_1 > div._endContents.c-heading-answer__content > div._endContentsText.c-heading-answer__content-user > div > div')
len(questions),len(divs)

InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00007FF6F4D078A2+54818]
	(No symbol) [0x00007FF6F4C76AD2]
	(No symbol) [0x00007FF6F4B2D8ED]
	(No symbol) [0x00007FF6F4B630DE]
	(No symbol) [0x00007FF6F4B8E9A2]
	(No symbol) [0x00007FF6F4B8A710]
	(No symbol) [0x00007FF6F4B8A016]
	(No symbol) [0x00007FF6F4B041D0]
	GetHandleVerifier [0x00007FF6F506B9A2+3610402]
	GetHandleVerifier [0x00007FF6F50C1870+3962352]
	GetHandleVerifier [0x00007FF6F50B9D5F+3930847]
	GetHandleVerifier [0x00007FF6F4DA3656+693206]
	(No symbol) [0x00007FF6F4C81638]
	(No symbol) [0x00007FF6F4B02FFE]
	GetHandleVerifier [0x00007FF6F5102288+4227080]
	BaseThreadInitThunk [0x00007FFFD4147344+20]
	RtlUserThreadStart [0x00007FFFD52E26B1+33]


In [7]:
answer = ''
for p in questions:
    text = p.get_text().strip()
    if text == '\u200b' or text == '':
        continue
    answer += text
print(answer + '\t' + 'hu')

마음에 드는 원룸이 보증금 5천 월세20 관리비 10입니다최우선변제금액이 3400만원이구요건물시세가 밸류맵으로 확인해봤을때 7억8천8백만원이였구요 등기부등본에 채권최고액이 5억4천으로공동담보로 잡혀있었습니다안전한 집 일까요…..	hu


In [8]:
div = divs[0]
ps = div.select('.se-text-paragraph.se-text-paragraph-align-')
len(ps)

4

In [9]:
answer = ''
for p in ps:
    text = p.get_text().strip()
    if text == '\u200b' or text == '':
        continue
    answer += text
print(answer)

상태로 보았을때 안전하지 않습니다.혹시 모를 위험애 대비해서 다른곳을 찾는걸 추천 드립니다..궁금증이 풀리셨다면 채택 부탁드립니다!


In [20]:
data_list = []
driver = webdriver.Chrome()
for page in range(1, 36):
    url = f'https://kin.naver.com/search/list.naver?query={quote("서울 월세")}&page={page}'
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.basic1 > li')

    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        driver.get(sub_url)
        time.sleep(2)

        sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
        questions = sub_soup.select('#content > div.question-content > div > div.c-heading._questionContentsArea.c-heading--default-old > div.c-heading__content')
        divs = sub_soup.select('#answer_1 > div._endContents.c-heading-answer__content > div._endContentsText.c-heading-answer__content-user > div > div')
        
        answer = ''
        for p in questions:
            text = p.get_text().strip()
            answer += text
        answer1 = ''
        for p in divs:
            text = p.get_text().strip()
            answer1 += text
        
        data_list.append({'구분': '월세', '질문': answer, '답변': answer1})
         
        driver.back()
        time.sleep(2)
    print()

driver.close()

The chromedriver version (117.0.5938.88) detected in PATH at C:\Users\human\Downloads\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (118.0.5993.88); currently, chromedriver 118.0.5993.70 is recommended for chrome 118.*, so it is advised to delete the driver in PATH and retry


In [21]:
df = pd.DataFrame(data_list)
df

,구분,질문,답변
0,월세,제 상황에 어떤 선택이 좋은지 모르겠어서 질문글 남깁니다.집 컨디션은 비슷한 것 같...,"월세가 차라리 더 좋아 보입니다.전체적으로 좀 싸게 들어가면 전세가 좋기는 하지만,..."
1,월세,서울 주거 포털에서 청년월세지원을 신청했는데 아직도 심사중이라고 뜹니다 월세지원 못...,결과 발표 11월중 예정이라고 돼 있어요
2,월세,,"안녕하세요. 답변드려요.​네, 서울 청년월세지원은 신청자에게 결과를 카톡으로 연락해..."
3,월세,인천 연수구 송도국제도로 123 오네스타 랑 최대한 가까운 곳으로 추천 부탁드립니다...,1.신림동.독산동.신월동.시흥동.가산동 등등 입니다그러나 보통 원룸 월세가 40~4...
4,월세,서울월세지원 10번 받아서 끝났는데 또 신청해서 또 받을수 있나요,답변드립니다최대 10개월 지원받고 200만원까지 가능한데 생애 1회만 가능합니다그러...
...,...,...,...
345,월세,이번에 하는 서울청년월세지원 쉐어하우스도 해당되나요?,"안녕하세요지식인 대출분야(담보대출,전세자금대출,신용대출,개인회생) 상담전문 뱅크솔루..."
346,월세,서울에서 자취중입니다. 1.5억 전세 원룸에 살고있는데 전세가 그대로 목적물변경으...,어 음...월세가 너무 부담 되지 않을까요?여건이 되고 지금 집이 괜찮다면 그냥 그...
347,월세,당장 오늘 집보러 가는데 아는게 없네요... 보통 월세 보증금이 5천 이상 하더라구요,서울 월세 집을 얻을때 알아둬야 할게 있을까요당장 오늘 집보러 가는데 아는게 없네요...
348,월세,16일부터 신청가능한 서울 청년월세지원 관련해서 메일이 왔는데 자격 조건이 월세 6...,안녕하세요. 서울생활 행복도우미 120다산콜센터 지식파트너입니다. ◈ 2020년 ...


In [23]:
df.to_csv('data/서울 월세.csv', index=False, encoding='utf-8')